# 实时任务提交

实时任务提交使用接口[Futures](https://docs.dask.org/en/latest/futures.html#api),它只能是使用集群部署才可以使用,它可以将任务直接提交到集群,然后等待结果,任务执行是异步的.

它的接口基本参照标准库`concurrent.futures`主要接口包括:

+ `submit`提交任务
+ `map`将任务分发到多个计算节点
+ `as_completed`构造一个生成器,Futures对象完成获取结果后yield出来
+ `wait`等待计算完成,可以设置超时

同时也提供了Queue,Lock,pub/sub,全局变量这样的工具,以应付数据同步这样的需求

实时任务提交主要用户构造服务和流数据处理.有点类似spark中spark-streaming的地位

## 同步提交

这种方式再README中就已经有演示,它和`concurrent.futures`几乎完全一样

In [1]:
from dask.distributed import Client
client = Client('localhost:8786')

def square(x):
    return x ** 2

A = client.map(square, range(10000))
total = client.submit(sum, A)
print(total.result())

333283335000


## 异步提交

dask的异步接口基于Tornado或者asyncio,默认使用的是Tornado,我们可以指定loop来指定使用什么事件循环.

### 在初始化`Client`对象时使用`asynchronous=True`,这样整个client将只有异步接口

In [4]:
%%writefile code/realtime/async_submit.py
import asyncio
import platform
import multiprocessing
from dask.distributed import Client


def square(x):
    return x ** 2


async def f():
    client = await Client("localhost:8786", asynchronous=True)
    A = client.map(square, range(10000))
    result = await client.submit(sum, A)
    print(result)
    await client.close()
    return result


if __name__ == '__main__':
    if "Windows" in platform.platform():
        multiprocessing.freeze_support()
    asyncio.get_event_loop().run_until_complete(f())

Overwriting code/realtime/async_submit.py


上面的方式我们使用的是基于进程池的`Client`对象,可以使用参数`processes=False`设定改为使用线程池的`Client`对象,这样就不需要在windows上使用`multiprocessing.freeze_support()`了

## Actor模式[实验]

这可能是dask下一个大版本的主要新特性,这个模式下我们的任务不走调度器,而是直接交给worker通过actor对象执行,这样的好处是减少了调度时间可以响应更快,但坏处是无法通过调度器自动优化,debug,而且一旦在worker上创建了actor,那除非worker关闭否则完全无法关闭.

Actor使用类定义,它可以快速的改变状态,我们可以像使用类实例一样的使用它,它同样也支持同步和异步两种方式

### 同步提交

In [9]:
class Counter:
    n = 0

    def __init__(self):
        self.n = 0

    def increment(self):
        self.n += 1
        return self.n

future = client.submit(Counter, actor=True)
counter = future.result()
counter

<Actor: Counter, key=Counter-ec9d78f8-8c50-4ae2-91d6-1ccfbd9e068c>

Actor模式的对象submit后返回的是`ActorFutures`类型,它可以调用定义好的方法,也可以访问其中的属性

In [10]:
future = counter.increment()
result = future.result()
result

1

In [11]:
counter.n

1

### 异步提交

这个模式下异步接口更加方便,上面的内容可以写成如下异步实现

In [5]:
%%writefile code/realtime/async_actor.py
import asyncio
import platform
import multiprocessing
from dask.distributed import Client


class Counter:
    n = 0

    def __init__(self):
        self.n = 0

    def increment(self):
        self.n += 1
        return self.n


async def f():
    client = await Client("localhost:8786", asynchronous=True)
    counter = await client.submit(Counter, actor=True)
    await counter.increment()
    n = await counter.n
    print(n)
    await client.close()
    return n


if __name__ == '__main__':
    if "Windows" in platform.platform():
        multiprocessing.freeze_support()
    asyncio.get_event_loop().run_until_complete(f())

Overwriting code/realtime/async_actor.py


上面的方式我们使用的是基于进程池的`Client`对象,可以使用参数`processes=False`设定改为使用线程池的`Client`对象,这样就不需要在windows上使用`multiprocessing.freeze_support()`了